In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [3]:
repo_path = "C:\\Users\\esmba\\OneDrive\\Documents\\CodeRepos\\codeinterpreter-api"

In [4]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()
len(documents)

35

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

60

In [21]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

role = os.getenv('AWS_SAGEMAKER_ROLE')

In [22]:
print(role)

arn:aws:iam::928680607065:role/SageMakerExecution


In [23]:
print(boto3.__version__)
print(sagemaker.__version__)

1.28.41
2.183.0


In [6]:
from huggingface_hub import HfApi
from huggingface_hub import ModelFilter
api = HfApi()

c:\Users\esmba\miniconda3\envs\ContextCoderEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
filter = ModelFilter(author= "WizardLM")
for model in api.list_models(filter=filter):
    print(model.modelId)

WizardLM/WizardLM-7B-V1.0
TheBloke/wizardLM-7B-HF
TheBloke/wizardLM-7B-GGML
TheBloke/wizardLM-7B-GPTQ
winddude/wizardLM-LlaMA-LoRA-7B
eachadea/ggml-wizardlm-7b
Aitrepreneur/wizardLM-7B-GPTQ-4bit-128g
winddude/wizardLM-LlaMA-LoRA-13bbbaaaaddd
ausboss/llama7b-wizardlm-unfiltered-lora
ausboss/llama7b-wizardlm-unfiltered
ausboss/llama7b-wizardlm-unfiltered-4bit-128g
winddude/wizardLM-LlaMA-LoRA-13
Bakanayatsu/llama7b-wizardlm-unfiltered-GGML
execveat/wizardLM-13b-ggml-4bit
ehartford/WizardLM-7B-Uncensored
TheBloke/WizardLM-7B-uncensored-GGML
TheBloke/WizardLM-7B-uncensored-GPTQ
4bit/WizardLM-7B-uncensored-GPTQ
ehartford/WizardLM-13B-Uncensored
TehVenom/WizardLM-13B-Uncensored-Q5_1-GGML
openaccess-ai-collective/mpt-7b-wizardlm
amaxen/WizardLM-13B-Uncensored
ausboss/wizardlm-13b-unbiased-lora
4bit/WizardLM-13B-Uncensored-4bit-128g
TehVenom/MPT-7b-WizardLM_Uncensored-Storywriter-Merge
mongolian-basket-weaving/wizardlm-13b-uncensored-ggml-f16
mongolian-basket-weaving/wizardlm-13b-uncensored-gg

In [30]:
# embed_repoid = 'WizardLM/WizardCoder-Python-34B-V1.0'
hf_API_key = os.getenv('HUGGINGFACE_API_TOKEN')

In [27]:
sess = sagemaker.Session()

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

print(f"role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")
print(f"S3 bucket: {sess.default_bucket()}")

role arn: arn:aws:iam::928680607065:role/SageMakerExecution
sagemaker session region: us-east-2
S3 bucket: sagemaker-us-east-2-928680607065


In [28]:
llm_image = get_huggingface_llm_image_uri(
    "huggingface",
    version = "0.9.3"
)

print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi0.9.3-gpu-py39-cu118-ubuntu20.04


In [31]:
# # Hub Model configuration. https://huggingface.co/models
# hub = {
# 	'HF_MODEL_ID':'WizardLM/WizardCoder-Python-34B-V1.0',
# 	'SM_NUM_GPUS': json.dumps(1),
#	'HUGGING_FACE_HUB_TOKEN': json.dumps(hf_API_key)
# }

# # create Hugging Face Model Class
# huggingface_model = HuggingFaceModel(
# 	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.9.3"),
# 	env=hub,
# 	role=role, 
# )

hub = {
	'HF_MODEL_ID':'Phind/Phind-CodeLlama-34B-v2',
	'SM_NUM_GPUS': json.dumps(1),
    'HUGGING_FACE_HUB_TOKEN': json.dumps(hf_API_key)
}


# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=llm_image,
	env=hub,
	role=role, 
)

In [32]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=3600,
  )

----------------------------------------------------------------------------------------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint huggingface-pytorch-tgi-inference-2023-09-08-16-52-59-029: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [ ]:
# send request
predictor.predict({
	"inputs": "Write a function that adds two numbers.",
})

In [21]:
from langchain.vectorstores import Chroma
from sagemaker_helper import ContentHandler
from langchain.embeddings import SagemakerEndpointEmbeddings

In [ ]:
db = Chroma.from_documents(texts, hf_embed)
retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k":4},
)

In [ ]:
content_handler = ContentHandler()

embeddings = SagemakerEndpointEmbeddings(
    # credentials_profile_name="credentials-profile-name",
    endpoint_name="WizardCoderEndpoint",
    region_name="us-east-1",
    content_handler=content_handler,
)

In [ ]:
query_result = embeddings.embed_query("for x in range(5):")
query_result

In [14]:
llm_repoid = 'WizardLM/WizardCoder-Python-34B-V1.0'

In [15]:
from langchain.llms import HuggingFaceHub
llm = HuggingFaceHub(repo_id=llm_repoid,huggingfacehub_api_token=hf_API_key) 

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)